# Diagnosisis on the computation of velocity and acceleration 

- [ ] velocity
- [ ] acceleration


In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd


#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import geopandas as gpd
from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt


import drifters.utils as ut
import pynsitu as pin

import dask.dataframe as dd
from dask.delayed import delayed

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/pyTMD/tools.py:55: UserWarning: ipyleaflet not available
  warnings.warn("ipyleaflet not available")
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/pyTMD/tools.py:56: UserWarning: Some functions will throw an exception if called
  warnings.warn("Some functions will throw an exception if called")


In [2]:
from dask.distributed import Client

if True:
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    #cluster = PBSCluster()
    #w = cluster.scale(jobs=3) # 2 not enough for lon, lat, year, binning
    cluster = PBSCluster(cores=3, processes=3, walltime='04:00:00')
    w = cluster.scale(jobs=8)
else:
    from dask.distributed import LocalCluster
    cluster = LocalCluster()
    
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.75:8787/status,
Dashboard: http://10.148.0.75:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.75:35623,Workers: 0
Dashboard: http://10.148.0.75:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
# load data

col_sel = ['typebuoy', 'gap',
           'deploy_date', 'deploy_lat','deploy_lon', 'end_date', 'end_lat', 'end_lon', 
           'drogue_lost_date','typedeath', 'lon360', 'err_lat',
           'err_lon', 'err_ve', 'err_vn']

bandwidth = 0.1
df_argos = ut.load_gdp("argos", suffix=f"_filtered_b{bandwidth}")
df_argos = df_argos.where(df_argos!=-1.e+34).reset_index().rename(columns=dict(index="id"))[col_sel].persist()

df_gps = ut.load_gdp("gps", suffix=f"_filtered_b{bandwidth}")
df_gps = (df_gps.where(df_gps!=-1.e+34).reset_index().rename(columns=dict(index="id")))[col_sel].persist() # ATTENTION : replace missing value -1.e+34 by Nan

In [4]:
# dev
#df_gps = dd.concat([df_gps.get_partition(n) for n in range(3)]).persist()
df = df_gps.get_partition(0).groupby('id').get_group(131969).compute()

## Computation : Version pynsitu
Derivation of velocities and acceleration :
- ve, vn already computed via LOWESS METHOD
- vex, vey centered derivation using projection of lon, lat
- aex, any double centered derivation using projection of lon, lat
- ae, an centered derivation of ve, vn

In [21]:
def c_vel_acc(df, **kwargs):
    _geo = pin.geo.GeoAccessor(df)
    df0 = _geo.compute_velocities(time="time", centered=True, v_name={'ve':'vex', 'vn':'vny', 'v':'vxy'})
    df1 = _geo.compute_acceleration(time='time', acc_name={'ae':'ae', 'an':'an', 'a':'aen'}, from_ = ('vevn','ve','vn'), centered_velocity=True, )
    df2 = _geo.compute_acceleration(time='time', acc_name={'ae':'aex', 'an':'any', 'a':'axy'}, from_ = ('lonlat','lon','lat') )
    del _geo
    return df0.merge(df1).merge(df2).merge(df[col_sel])

In [23]:
df_va = c_vel_acc(df)
df_va

,time,id,lon,lat,vex,vny,vxy,ve,vn,ae,...,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn
0,2014-02-18 20:00:00,131969,0.72649,38.723270,-0.028510,-0.419881,0.420848,-0.0298,-0.3110,1.305555e-06,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72649,0.00048,0.00062,0.0182,0.0182
1,2014-02-18 21:00:00,131969,0.72519,38.713181,-0.028510,-0.419881,0.420848,-0.0320,-0.3121,1.305555e-06,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72519,0.00076,0.00098,0.0181,0.0181
2,2014-02-18 22:00:00,131969,0.72413,38.705109,-0.025493,-0.359729,0.360631,-0.0204,-0.2998,3.277778e-06,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72413,0.00064,0.00081,0.0150,0.0150
3,2014-02-18 23:00:00,131969,0.72308,38.697090,-0.025375,-0.358201,0.359099,-0.0084,-0.2873,3.319445e-06,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72308,0.00052,0.00066,0.0117,0.0117
4,2014-02-19 00:00:00,131969,0.72203,38.689072,-0.025378,-0.358258,0.359156,0.0035,-0.2747,3.319444e-06,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72203,0.00044,0.00057,0.0091,0.0091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,2014-09-13 05:00:00,131969,1.51884,38.679611,0.000363,0.001477,0.001521,0.0000,0.0016,1.805556e-07,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51884,0.00037,0.00047,0.0137,0.0137
4552,2014-09-13 06:00:00,131969,1.51889,38.679630,0.001209,-0.000017,0.001209,0.0012,-0.0016,3.333333e-07,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51889,0.00050,0.00064,0.0184,0.0184
4553,2014-09-13 07:00:00,131969,1.51894,38.679630,0.001570,0.000161,0.001579,0.0024,0.0016,0.000000e+00,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51894,0.00037,0.00047,0.0137,0.0137
4554,2014-09-13 08:00:00,131969,1.51902,38.679680,0.001933,0.000857,0.002115,0.0012,0.0017,0.000000e+00,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51902,0.00055,0.00071,0.0185,0.0185


In [24]:
df_gps_out = df_gps.groupby("id").apply(c_vel_acc).set_index('time')
df_argos_out = df_argos.groupby("id").apply(c_vel_acc).set_index('time')

In [25]:
df_gps_out = df_gps_out.persist()
df_argos_out = df_argos_out.persist()

# Store in parquet

In [26]:
root_dir = "/home1/datawork/mdemol/GDP"
parquet_argos = os.path.join(root_dir, "argos_av_time.parquet")
parquet_gps = os.path.join(root_dir, "gps_av_time.parquet")

In [27]:
df_load_gps = (df_gps_out.reset_index().repartition(partition_size="100MB").persist())
df_load_argos = (df_argos_out.reset_index().repartition(partition_size="100MB").persist())

In [28]:
df_load_gps.to_parquet(parquet_gps, engine='pyarrow') # alternative: fastparquet
df_load_argos.to_parquet(parquet_argos, engine='pyarrow') # alternative: fastparquet

In [29]:
df_gps_reload = dd.read_parquet(parquet_gps)

In [30]:
df_gps_reload.get_partition(0).compute()

,time,id,lon,lat,vex,vny,vxy,ve,vn,ae,...,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn
0,2001-07-01 02:00:00,22192,-94.795609,5.537910,0.649771,-0.761679,1.001178,0.6460,-0.2900,-2.777825e-07,...,2001-12-19,-3.90,-121.28,2001-12-19,3.0,265.204376,0.00052,0.00052,0.0100,0.0100
1,2001-07-01 03:00:00,22192,-94.774597,5.528520,0.649771,-0.761679,1.001178,0.6466,-0.2901,-2.777825e-07,...,2001-12-19,-3.90,-121.28,2001-12-19,3.0,265.225403,0.00052,0.00052,0.0134,0.0134
2,2001-07-01 04:00:00,22192,-94.753387,5.518760,0.656827,-0.785170,1.023676,0.6440,-0.2925,-1.361113e-06,...,2001-12-19,-3.90,-121.28,2001-12-19,3.0,265.246613,0.00065,0.00065,0.0135,0.0135
3,2001-07-01 05:00:00,22192,-94.731918,5.508410,0.660713,-0.807147,1.043085,0.6368,-0.2986,-2.000001e-06,...,2001-12-19,-3.90,-121.28,2001-12-19,3.0,265.268066,0.00059,0.00059,0.0122,0.0122
4,2001-07-01 06:00:00,22192,-94.710457,5.498070,0.660724,-0.806720,1.042762,0.6296,-0.3047,-2.000001e-06,...,2001-12-19,-3.90,-121.28,2001-12-19,3.0,265.289551,0.00053,0.00053,0.0109,0.0109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206376,2008-07-05 14:00:00,2020710,-36.441422,-37.010651,0.108405,-0.122165,0.163328,0.1174,-0.0398,4.166668e-06,...,2009-10-01,-31.58,-36.15,2008-10-09,3.0,323.558594,0.00037,0.00047,0.0046,0.0046
206377,2008-07-05 14:00:00,2024710,-44.958729,-34.865330,-0.005329,-0.020435,0.021118,-0.0030,-0.0180,2.083333e-07,...,2008-12-28,-39.18,-50.49,2008-12-28,3.0,315.041260,0.00037,0.00046,0.0046,0.0046
206378,2008-07-05 14:00:00,2527260,-43.389420,-34.778561,-0.037341,-0.100528,0.107239,-0.0385,-0.0846,1.555556e-06,...,2008-07-30,-34.60,-42.67,2007-12-31,3.0,316.610565,0.00037,0.00045,0.0046,0.0046
206379,2008-07-05 14:00:00,2027690,-39.354870,-37.667099,0.013276,0.088214,0.089208,0.0135,0.0593,-5.555559e-08,...,2008-12-21,-34.94,-36.69,2008-09-08,3.0,320.645142,0.00037,0.00047,0.0046,0.0046


In [31]:
df_gps_reload.columns

Index(['time', 'id', 'lon', 'lat', 'vex', 'vny', 'vxy', 've', 'vn', 'ae', 'an',
       'aen', 'aex', 'any', 'axy', 'typebuoy', 'gap', 'deploy_date',
       'deploy_lat', 'deploy_lon', 'end_date', 'end_lat', 'end_lon',
       'drogue_lost_date', 'typedeath', 'lon360', 'err_lat', 'err_lon',
       'err_ve', 'err_vn'],
      dtype='object')

In [32]:
cluster.close()

distributed.scheduler - ERROR - Error transitioning "('reset_index-790ef5c1bf23266424fab3b127b7b93a', 57)" from 'processing' to 'erred'
Traceback (most recent call last):
  File "/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/distributed/scheduler.py", line 1809, in _transition
    recommendations, client_msgs, worker_msgs = func(
TypeError: transition_processing_erred() missing 1 required positional argument: 'worker'
distributed.scheduler - ERROR - transition_processing_erred() missing 1 required positional argument: 'worker'
Traceback (most recent call last):
  File "/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/distributed/utils.py", line 742, in wrapper
    return await func(*args, **kwargs)
  File "/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/distributed/scheduler.py", line 4829, in remove_worker
    self.transitions(recommendations, stimulus_id=stimulus_id)
  File "/home1/datahome/mdemol/.minicon